# Switzerland Graph Construction Story

This notebook visualizes the step-by-step process of transforming raw Swisstopo data into a connected NetworkX graph.

**Steps:**
1.  **Raw Data**: Comparing SBB Data (Red) vs Swisstopo (Gray).
2.  **Node Processing**: Identifying which nodes are actual stations.
3.  **Graph Construction**: Linking these components.
4.  **Component Analysis**: Visualizing network connectivity.

In [ ]:
%load_ext autoreload
%autoreload 2
import importlib
import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import geopandas as gpd

# Add project root to path to import src modules
project_root = Path("../../").resolve()
sys.path.append(str(project_root))

# Create maps directory
os.makedirs("maps/switzerland", exist_ok=True)

from src.processing.switzerland import load_data, get_station_abbreviations, build_graph

import src.processing.visualize as visualize
importlib.reload(visualize)
from src.processing import create_folium_map, plot_connected_components, create_component_map

## 1. Load Raw Data (SBB vs Swisstopo)
We start by inspecting the **SBB Dataset** (Legacy) and comparing it to the **Swisstopo Dataset** (Nationwide).

In [ ]:
base_dir = project_root / "datasets/switzerland"
station_metadata, net_segments, net_nodes = load_data(base_dir)

# SBB Stations (for comparison)
sbb_stations = station_metadata[station_metadata['stopPoint'].astype(str).str.lower() == 'true']
print(f"SBB Stations (Legacy): {len(sbb_stations)}")

# Swisstopo Data
print(f"Swisstopo Nodes: {len(net_nodes)}")
print(f"Swisstopo Segments: {len(net_segments)}")

### Visualization: SBB Coverage
Comparing the two datasets on an interactive map. Red = SBB Stations, Gray Lines = Swisstopo Rail.

In [ ]:

import os
import folium
import math
import pickle
import geopandas as gpd
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output
from src.processing.visualize import create_static_map

# ----------------- DATA PREP -----------------
# 1. Swisstopo Data
if 'net_segments' in locals():
    net_segments_wgs84 = net_segments.to_crs(4326)
else:
    net_segments_wgs84 = None
if 'net_nodes' in locals():
    net_nodes_wgs84 = net_nodes.to_crs(4326)
else:
    net_nodes_wgs84 = None

# 2. SBB Data (Load Pickle)
sbb_graph_path = Path("../datasets/switzerland/sbb_rail_network.gpickle")
G_sbb = None
if sbb_graph_path.exists():
    with open(sbb_graph_path, 'rb') as f:
        G_sbb = pickle.load(f)

# ----------------- MAP FUNCTIONS -----------------
def create_swisstopo_map():
    m = folium.Map(location=[46.8182, 8.2275], zoom_start=8, tiles='CartoDB Positron', prefer_canvas=True)
    if net_segments_wgs84 is not None:
        edges_layer = folium.FeatureGroup(name="Swisstopo Edges")
        lines = []
        for geom in net_segments_wgs84.geometry:
            if geom and not geom.is_empty:
                coord_list = []
                if geom.geom_type == 'LineString': coord_list.append(geom)
                elif geom.geom_type == 'MultiLineString': coord_list.extend(geom.geoms)
                for part in coord_list:
                    coords = [(y, x) for x, y in part.coords if not (math.isnan(x) or math.isnan(y))]
                    if coords: lines.append(coords)
        if lines:
             folium.PolyLine(lines, color='gray', weight=2, opacity=0.6).add_to(edges_layer)
        edges_layer.add_to(m)

    if net_nodes_wgs84 is not None:
        nodes_layer = folium.FeatureGroup(name="Swisstopo Nodes")
        for idx, row in net_nodes_wgs84.iterrows():
            if row.geometry and not row.geometry.is_empty:
                y, x = row.geometry.y, row.geometry.x
                if not (math.isnan(y) or math.isnan(x)):
                     folium.Circle([y,x], radius=15, color='#3388ff', fill=True, fill_opacity=0.8, weight=0, tooltip="Swisstopo Node").add_to(nodes_layer)
        nodes_layer.add_to(m)
    return m

def create_sbb_map():
    m = folium.Map(location=[46.8182, 8.2275], zoom_start=8, tiles='CartoDB Positron', prefer_canvas=True)
    if G_sbb:
        edges_layer = folium.FeatureGroup(name="SBB Edges")
        nodes_layer = folium.FeatureGroup(name="SBB Nodes")
        positions = {}
        for n, data in G_sbb.nodes(data=True):
            lat, lon = data.get('lat'), data.get('lon')
            if lat is not None and lon is not None: positions[n] = (lat, lon)
            
        lines = []
        for u, v in G_sbb.edges():
            if u in positions and v in positions: lines.append([positions[u], positions[v]])
        if lines:
            folium.PolyLine(lines, color='#6c757d', weight=2, opacity=0.5).add_to(edges_layer)
        edges_layer.add_to(m)
        
        for n, pos in positions.items():
             folium.Circle(pos, radius=15, color='#e6194b', fill=True, fill_opacity=0.8, weight=0, tooltip=str(n)).add_to(nodes_layer)
        nodes_layer.add_to(m)
    return m

# ----------------- UI CONTROL -----------------
if os.environ.get('CI'):
    print("CI detected: Generating static map via Matplotlib for GitHub compatibility.")
    # Static fallback for Swisstopo data (reconstructing a graph for visualization)
    # Since Swisstopo data is GeoDataFrame here, we can plot it directly with matplotlib or try to convert to graph
    # But simpler: display the SBB graph if available, or just plot the GDFS
    if G_sbb:
         create_static_map(G_sbb, title='SBB Network (Static)')
    else:
         if net_segments_wgs84 is not None:
             ax = net_segments_wgs84.plot(color='gray', alpha=0.5, figsize=(10,10))
             if net_nodes_wgs84 is not None:
                 net_nodes_wgs84.plot(ax=ax, color='blue', markersize=10)
             import matplotlib.pyplot as plt
             plt.axis('off')
             plt.title('Swisstopo Network (Static)')
             plt.show()
else:
    # ----------------- UI LOGIC -----------------
    selector = widgets.RadioButtons(
        options=['Swisstopo (Nationwide Graph)', 'SBB (Legacy Graph)'],
        value='Swisstopo (Nationwide Graph)',
        description='Dataset:',
        disabled=False
    )

    out = widgets.Output()

    def on_change(change):
        # STRICT Filtering: Only proceed if the 'value' changed
        if change['name'] == 'value' and change['type'] == 'change':
            with out:
                clear_output() # Wipe perfectly
                if 'Swisstopo' in change['new']:
                    display(create_swisstopo_map())
                else:
                    display(create_sbb_map())

    # Attach observer
    selector.observe(on_change)

    # Initial Display
    display(selector)
    display(out)
    with out:
        clear_output()
        display(create_swisstopo_map())


In [ ]:
station_abbreviation_set = get_station_abbreviations(station_metadata)
G = build_graph(net_nodes, net_segments, station_abbreviation_set)

## 3. Connected Components Analysis
Before we finish, let's verify if the graph is fully connected. Ideally, all rails in Switzerland should be reachable.

## 2. Graph Construction (Swisstopo)
Now we switch to building the full graph using Swisstopo data.

In [ ]:
plot_connected_components(G, title="Swisstopo Connected Components")

### Interactive Component Map
Explore the disconnected islands interactively.

In [ ]:
m = create_component_map(G)
output_path = "maps/switzerland/swiss_component_map.html"
m.save(output_path)
print(f"Map saved to {output_path}. If it doesn't appear below, open this file manually.")
m

## 4. Final Interactive Map
The fully processed graph, scrollable and zoomable.

In [ ]:
# LOAD THE UNIFIED DATASET (The 'One True Graph')
# This ensures we visualize the final cleaned version, not the intermediate construction variable 'G'
import pickle
import os
from src.analysis.config import AnalysisConfig
from src.analysis.visualizer import NetworkVisualizer
from IPython.display import display

# Notebook is in a subdir, so we need to go up one level to find datasets
unified_path_rel = AnalysisConfig.get_graph_path('switzerland')
unified_path = project_root / unified_path_rel
print(f"Loading Unified Graph from: {unified_path}")

with open(unified_path, 'rb') as f:
    G_unified = pickle.load(f)

viz = NetworkVisualizer()
# Create the static component map (Blue Core / Red Isolated) with large visible nodes
m = viz.create_component_map(G_unified)
display(m)
